In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import climetlab as cml
import climetlab_s2s_ai_challenge
import pathlib
import xarray as xr

## Get raw obs from renku

In [ ]:
print(f'Climetlab version : {cml.__version__}')
print(f'Climetlab-s2s-ai-challenge plugin version : {climetlab_s2s_ai_challenge.__version__}')

In [ ]:
cml.settings.get('cache-directory')

In [ ]:
cmlds = cml.load_dataset("s2s-ai-challenge-observations", parameter=['t2m', 'pr'])

In [ ]:
organizers = cmlds.to_xarray()

In [ ]:
#renku.to_netcdf('***BASEDIR***/from-climetlab/observations.nc')

In [ ]:
organizers.sel(time=organizers.time.dt.year == 2010).to_netcdf('***BASEDIR***/organizers2010.nc')

## Get arlan data

In [ ]:
ARLAN_DATA_DIR = '***BASEDIR***/arlan-data'

In [ ]:
t2m_files = sorted(list(pathlib.Path(ARLAN_DATA_DIR).glob('2m*.nc')))
pr_files = sorted(list(pathlib.Path(ARLAN_DATA_DIR).glob('total*.nc')))

In [ ]:
def preprocess_t2m(t2m_dataset):
    return t2m_dataset.rename(lead_time='time')

In [ ]:
sample = preprocess_t2m(xr.open_dataset(files[0]))

In [ ]:
arlan_t2m = xr.open_mfdataset(t2m_files, preprocess=preprocess_t2m).rename(fill_flag='t2m_fill_flag')
arlan_tp = xr.open_mfdataset(pr_files).rename(fill_flag='pr_fill_flag', precip='pr')

In [ ]:
arlan = xr.merge([arlan_t2m, arlan_tp])

In [ ]:
arlan.to_netcdf('***BASEDIR***/arlan-data/merged.nc')

In [ ]:
arlan.sel(time=arlan.time.dt.year == 2010).to_netcdf('***BASEDIR***/arlan2010.nc')

## Compare a bit

In [ ]:
t2m_delta = arlan.t2m - organizers.t2m.sel(time=arlan.time)

In [ ]:
(t2m_delta**2).max(dim='time').plot()

In [ ]:
pr_delta = arlan.pr - organizers.pr.sel(time=arlan.time)

In [ ]:
(pr_delta**2).max(dim='time').plot()

## Compare with terciles we get

In [ ]:
edges = xr.open_dataset('***BASEDIR***2/from-template-lfs/hindcast-like-observations_2000-2019_biweekly_tercile-edges.nc')

In [ ]:
edges

In [ ]:
terciles = xr.open_dataset('***BASEDIR***2/from-template-lfs/hindcast-like_observations_2000-2019_biweekly_terciled.nc')

In [ ]:
terciles